# How to run a docker process background and nohup

* [Why does running a background task over ssh fail if a pseudo-tty is allocated?](https://stackoverflow.com/a/32387290/4281353)

> This is related with how process groups work, how bash behaves when invoked as a non-interactive shell with ```-c```, and **the effect of ```&``` in input commands**. The answer assumes you're familiar with **how job control works in UNIX**. If the concept of process groups and job control is completely new to you, I think you'll need to read up on that to fully understand this answer. A great resource to learn this is Chapter 9 of Advanced Programming in the UNIX Environment (3rd edition).
> 
> ### When you don't use -t
> 
> When you don't use -t, there is no PTY allocation on the remote side, so bash is not a session leader, and in fact no new session is created. Because sshd is running as a daemon, the bash process that is forked + exec()'d will not have a controlling terminal. As such, even though the shell terminates very quickly (probably before touch(1)), there is no SIGHUP sent to the process group, because bash wasn't a session leader (and there is no controlling terminal). So everything works.
> 
> ### When you use -t
> 
> -t forces PTY allocation, which means that the ssh remote side will call setsid(2), allocate a pseudo-terminal + fork a new process with forkpty(3), connect the PTY master device input and output to the socket endpoints that lead to your machine, and finally execute bash(1). forkpty(3) opens the PTY slave side in the forked process that will become bash; since there's no controlling terminal for the current session, and a terminal device is being opened, the PTY device becomes the controlling terminal for the session and bash becomes the session leader.
> 
> Then the same thing happens again: touch(1) is executed in the same process group, etc., yadda yadda. The point is, this time, there is a session leader and a controlling terminal. So, since bash does not bother waiting because of the &, when it exits, SIGHUP is delivered to the process group and touch(1) dies prematurely.
> 
> About nohup
> 
> nohup(1) doesn't work here because there is still a race condition. If bash(1) terminates before nohup(1) has the chance to set up the necessary signal handling and file redirection, it will have no effect (which is probably what happens)

* [How to get docker commands to run in the background with nohup](https://stackoverflow.com/a/63601442/4281353)

> removing both the -i and -t flags would probably be sufficient
> ```
> --interactive , -i      Keep STDIN open even if not attached
> --tty , -t              Allocate a pseudo-TTY
> ```

In [ ]:
# DO NOT use -i -t
nohup docker run --rm ${IMAGE}:${TAG} > run.log 2>&1 &